In [1]:
#
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import re
import random

In [2]:
chars1 = list(string.ascii_lowercase)
chars2 = list(string.ascii_lowercase)

random.shuffle(chars2) #Inittiate randonly

true_mapping = {}  # true_mapping[encripted] = decripted

for i, letter in enumerate(chars1):
    if letter not in true_mapping:
        true_mapping[letter] = chars2[i]
    else:
        raise Exception("Repeated letters")

true_mapping

{'a': 'u',
 'b': 'e',
 'c': 's',
 'd': 'a',
 'e': 'l',
 'f': 'r',
 'g': 't',
 'h': 'b',
 'i': 'g',
 'j': 'w',
 'k': 'n',
 'l': 'f',
 'm': 'z',
 'n': 'k',
 'o': 'i',
 'p': 'q',
 'q': 'v',
 'r': 'c',
 's': 'x',
 't': 'o',
 'u': 'p',
 'v': 'm',
 'w': 'h',
 'x': 'j',
 'y': 'd',
 'z': 'y'}

In [3]:
#char2idx -> decoder/encoder
char2idx = {}
idx2char = []
for i, char in enumerate(chars1):
    char2idx[char] = i
    idx2char.append(char)

In [4]:
direct = '../input/text-dataset/'
file = 'moby_dick.txt'


with open(direct+file, 'r') as file:
    text = file.read()

In [5]:
#Lets create a class to receive a input text and populate the language model:

class LanguageModel():
    
    def __init__(self,text,vocab):
        
        words_list = self._cleanText(text)
        self.vocab = vocab

        self._MarkovModel(words_list)
        self._encoder_decoder()
    
    def _encoder_decoder(self):
        
        #char2idx -> decoder/encoder
        self.char2idx = {}
        self.idx2char = []
        for i, char in enumerate(self.vocab):
            self.char2idx[char] = i
            self.idx2char.append(char)
    
        
    def _cleanText(self,text):
        '''
            Input: text[str] -> text that will be used to populate the language model.
            All non-alph characters will be removed
            
            Output: words_list[list of strings] -> all words from the text
            
        '''
        pattern = re.compile('[^a-zA-Z]') #Re for non-alpha
        clean_txt = pattern.sub(' ',text).lower()
        words_list = clean_txt.split()
        return words_list
    
    
    def _MarkovModel(self, words_list):
        #Populate Language Model - Markov Model p(x1) ; p(x_n|x_{n-1})
        self.pi_matrix = np.ones(len(chars1))
        self.A_matrix = np.ones((len(chars1),len(chars1)))

        for word in words_list:
            self.pi_matrix[char2idx[word[0]]] += 1

            for i in range(1, len(word)):
                self.A_matrix[ char2idx[word[i-1]] , char2idx[word[i]]  ] +=1

        #Normalize pi and A
        self.pi_matrix = self.pi_matrix/np.sum(self.pi_matrix)
        self.A_matrix = self.A_matrix/self.A_matrix.sum(axis = 1, keepdims = True) #Normalize with respect to the line
        
    def log_prob_word(self, word):
        logprob = np.log(self.pi_matrix[self.char2idx[word[0]]])
        
        for i in range(1,len(word)):
            logprob+= np.log(  self.A_matrix[self.char2idx[word[i-1]], self.char2idx[word[i]]]  )
                       
        return logprob
    
    def log_prob_text(self, word_list):
        logprob = 0
        for word in word_list:
            logprob+=self.log_prob_word(word)
            
        return logprob
        

In [6]:
lang_model = LanguageModel(text, list(string.ascii_lowercase))

In [7]:
lang_model.A_matrix[1,:].sum()

1.0

In [8]:
lang_model.pi_matrix.sum()

1.0

In [9]:
lang_model.log_prob_word('cat')

-7.134580326834801

In [10]:
lang_model.log_prob_word('car')

-7.40615519582046

In [11]:
lang_model.log_prob_word('house')

-11.050122943513285

In [12]:
lang_model.log_prob_word('htsws')

-18.09994866591706

## Cipher Section:

In [13]:
original_message = '''I then lounged down the street and found,
as I expected, that there was a mews in a lane which runs down
by one wall of the garden. I lent the ostlers a hand in rubbing
down their horses, and received in exchange twopence, a glass of
half-and-half, two fills of shag tobacco, and as much information
as I could desire about Miss Adler, to say nothing of half a dozen
other people in the neighbourhood in whom I was not in the least
interested, but whose biographies I was compelled to listen to.
'''

In [14]:
def encrypt_msg(text, true_mapping):
    chars = []   
    for i, char in enumerate(text):
        if char.isupper():
            chars.append(true_mapping.get(char.lower(), char.lower()).upper())
        
        else:
            chars.append(true_mapping.get(char, char))
        
        
    return ''.join(chars)

In [15]:
encrypted_msg = encrypt_msg(original_message, true_mapping)
encrypted_msg

'G oblk fipktla aihk obl xocllo uka ripka,\nux G ljqlsola, obuo oblcl hux u zlhx gk u fukl hbgsb cpkx aihk\ned ikl huff ir obl tucalk. G flko obl ixoflcx u buka gk cpeegkt\naihk oblgc bicxlx, uka clslgmla gk ljsbuktl ohiqlksl, u tfuxx ir\nbufr-uka-bufr, ohi rgffx ir xbut oieussi, uka ux zpsb gkriczuogik\nux G sipfa alxgcl ueipo Zgxx Uaflc, oi xud kiobgkt ir bufr u aiylk\nioblc qliqfl gk obl klgtbeipcbiia gk hbiz G hux kio gk obl fluxo\ngkolclxola, epo hbixl egitcuqbglx G hux sizqlffla oi fgxolk oi.\n'

In [16]:
#Get words from crypted_msg
pattern = re.compile('[^a-zA-Z]') #Re for non-alpha
clean_txt = pattern.sub(' ',encrypted_msg).lower()
word_list_crypt = clean_txt.split()

clean_txt = pattern.sub(' ',original_message).lower()
word_list_original = clean_txt.split()

print(list(zip(word_list_original[:10],word_list_crypt[:10])))

[('i', 'g'), ('then', 'oblk'), ('lounged', 'fipktla'), ('down', 'aihk'), ('the', 'obl'), ('street', 'xocllo'), ('and', 'uka'), ('found', 'ripka'), ('as', 'ux'), ('i', 'g')]


In [17]:
lang_model.log_prob_text(word_list_original)   #Check log prob of original list

-933.0339420615785

In [18]:
lang_model.log_prob_text(word_list_crypt)   # Check log prob of encrypted list

-1888.6423158694145

In [19]:
def decrypet_msg(word_list, mapping, char2idx):
    word_list_output = []
    for i, text in enumerate(word_list):
        chars = []
        for char in text:
            chars.append(mapping[char2idx[char]])
        word_list_output.append(''.join(chars))
    return word_list_output

## Genetic Algorithm to find the best mapping

In [20]:
#Function to change the values of two positions of a list
def shuffle_list(lst, pos1, pos2):
    lst_output = lst.copy()
    lst_output[pos1] = lst[pos2]
    lst_output[pos2] = lst[pos1]
    
    return lst_output


In [21]:
# Define first generation size
n_0 = 300
n_children = 3 #Define how many childrens each parent will have
n_parent = n_0

chars = list(string.ascii_lowercase)

#Populate first generation mapping
mapping_list = []
for _ in range(n_0):
    random.shuffle(chars)
    mapping_list.append(chars.copy())
    
#Get first generation childrens
children_mapping = []

for i in range(n_parent):    
    for ch in range(n_children):      
        pos1, pos2 = random.sample(range(len(mapping_list[i])), 2)
        children_mapping.append(shuffle_list(mapping_list[i], pos1, pos2))
    #Create childrens

mapping_list.extend(children_mapping)

In [22]:
#Calculate the probabilities from each map

log_probs = np.zeros(len(mapping_list))

for i in range(len(mapping_list)):
    
    #Decrypet text:
    decryp_msg = decrypet_msg(word_list_crypt, mapping_list[i], lang_model.char2idx)
    log_probs[i] = (lang_model.log_prob_text(decryp_msg))

sorted_log = np.argsort(-log_probs)
max_log = max(log_probs)

In [23]:
#Lets iterate
error = 1e50
counter_ite = 0
while counter_ite<100:
    #New parents:
    max_idx = sorted_log[:n_parent]
    mapping_list = [mapping_list[idx] for idx in max_idx]
    
    #New childrens
    children_mapping = []
    for i in range(n_parent):    
        for ch in range(n_children):      
            pos1, pos2 = random.sample(range(len(mapping_list[i])), 2)
            children_mapping.append(shuffle_list(mapping_list[i], pos1, pos2))
        #Create childrens

    mapping_list.extend(children_mapping)
    
    log_probs = np.zeros(len(mapping_list))
    for i in range(len(mapping_list)):
        #Decrypet text:
        decryp_msg = decrypet_msg(word_list_crypt, mapping_list[i], lang_model.char2idx)
        log_probs[i] = (lang_model.log_prob_text(decryp_msg))
    
    sorted_log = np.argsort(-log_probs)
    max_log_new = max(log_probs)
    error = (max_log_new - max_log)/max_log
    max_log = max_log_new
   
    counter_ite+=1
    if counter_ite%20 == 0:
        print(f"Iteration: {counter_ite}\nMax logprob: {max(log_probs)}\nError: {error}")
   
    
    #Output:
    #Iteration: 20
    #Max logprob: -1301.261678639069
    
    #Iteration: 40
    #Max logprob: -1239.0814625491487
    
    #Iteration: 60
    #Max logprob: -1124.135395861417
    
    #Iteration: 80
    #Max logprob: -1076.411280501641
    
    #Iteration: 100
    #Max logprob: -1027.3659615796932


Iteration: 20
Max logprob: -1153.0964402600894
Error: -0.0
Iteration: 40
Max logprob: -993.4812712900668
Error: -0.0
Iteration: 60
Max logprob: -945.2745851900379
Error: -0.0
Iteration: 80
Max logprob: -929.9518049377775
Error: -0.0
Iteration: 100
Max logprob: -929.5929889514528
Error: -0.0


#A better way to converge the results would be to acelerate the propagation of the best genes. Lets try a different approach:
#Lets iterate
error = 1e50
counter_ite = 0
while counter_ite<100:
    #New parents:
    max_idx = sorted_log[:n_parent]
    mapping_list = [mapping_list[idx] for idx in max_idx]
    
    #Repeat best gene in 10% of the population
    n_repeat = len(mapping_list)//10
    best_mapping = [mapping_list[0]]*(n_repeat)
    mapping_list = best_mapping + mapping_list[:-n_repeat].copy()
    
    #New childrens
    children_mapping = []
    for i in range(n_parent):    
        for ch in range(n_children):      
            pos1, pos2 = random.sample(range(len(mapping_list[i])), 2)
            children_mapping.append(shuffle_list(mapping_list[i], pos1, pos2))
        #Create childrens

    mapping_list.extend(children_mapping)
    
    log_probs = np.zeros(len(mapping_list))
    for i in range(len(mapping_list)):
        #Decrypet text:
        decryp_msg = decrypet_msg(word_list_crypt, mapping_list[i], lang_model.char2idx)
        log_probs[i] = (lang_model.log_prob_text(decryp_msg))
    
    sorted_log = np.argsort(-log_probs)
    
    counter_ite+=1
    if counter_ite%20 == 0:
        print(f"Iteration: {counter_ite}\nMax logprob: {max(log_probs)}")


In [24]:
#Final output
max_idx = sorted_log[0]

final_mapping = mapping_list[max_idx]

decryp_msg = decrypet_msg(word_list_crypt,final_mapping , lang_model.char2idx)
print(decryp_msg)

['i', 'then', 'lounged', 'down', 'the', 'street', 'and', 'found', 'as', 'i', 'expected', 'that', 'there', 'was', 'a', 'mews', 'in', 'a', 'lane', 'which', 'runs', 'down', 'by', 'one', 'wall', 'of', 'the', 'garden', 'i', 'lent', 'the', 'ostlers', 'a', 'hand', 'in', 'rubbing', 'down', 'their', 'horses', 'and', 'received', 'in', 'exchange', 'twopence', 'a', 'glass', 'of', 'half', 'and', 'half', 'two', 'fills', 'of', 'shag', 'tobacco', 'and', 'as', 'much', 'information', 'as', 'i', 'could', 'desire', 'about', 'miss', 'adler', 'to', 'say', 'nothing', 'of', 'half', 'a', 'doken', 'other', 'people', 'in', 'the', 'neighbourhood', 'in', 'whom', 'i', 'was', 'not', 'in', 'the', 'least', 'interested', 'but', 'whose', 'biographies', 'i', 'was', 'compelled', 'to', 'listen', 'to']


In [25]:
#Compare text
print(final_mapping)

print([true_mapping[char] for char in final_mapping])

['d', 'h', 'r', 'y', 'b', 'l', 'i', 'w', 'o', 'x', 'n', 'e', 'v', 'z', 't', 'u', 'p', 'f', 'c', 'g', 'a', 'q', 'j', 's', 'k', 'm']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'y', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'n', 'z']


In [26]:
print(true_mapping)

{'a': 'u', 'b': 'e', 'c': 's', 'd': 'a', 'e': 'l', 'f': 'r', 'g': 't', 'h': 'b', 'i': 'g', 'j': 'w', 'k': 'n', 'l': 'f', 'm': 'z', 'n': 'k', 'o': 'i', 'p': 'q', 'q': 'v', 'r': 'c', 's': 'x', 't': 'o', 'u': 'p', 'v': 'm', 'w': 'h', 'x': 'j', 'y': 'd', 'z': 'y'}
